In [2]:
import sys
sys.path.append("/home/julia/USF/spring2/productAnalytics/facenet/src")

from align import detect_face
from facenet import prewhiten, crop, flip, to_rgb
import facenet
import tensorflow as tf
from scipy import misc
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm 
import os
import numpy as np
import pickle

def load_and_align_data(image_paths, image_size=160, margin=44):

    minsize = 20 # minimum size of face
    threshold = [ 0.6, 0.7, 0.7 ]  # three steps's threshold
    factor = 0.709 # scale factor

    print('Creating networks and loading parameters')
    with tf.Graph().as_default():
        sess = tf.Session()
        with sess.as_default():
            pnet, rnet, onet = detect_face.create_mtcnn(sess, None)

    img_list = []
    nrof_samples = len(image_paths)
    success_paths = []
    for img_path in tqdm(image_paths):
        img = misc.imread(img_path)
        img_size = np.asarray(img.shape)[0:2]
        bounding_boxes, _ = detect_face.detect_face(img, minsize, pnet, rnet, onet, threshold, factor)
        if len(bounding_boxes) < 1:
            print("no face found in {}".format(img_path))
            continue
        det = np.squeeze(bounding_boxes[0,0:4])
        bb = np.zeros(4, dtype=np.int32)
        bb[0] = np.maximum(det[0]-margin/2, 0)
        bb[1] = np.maximum(det[1]-margin/2, 0)
        bb[2] = np.minimum(det[2]+margin/2, img_size[1])
        bb[3] = np.minimum(det[3]+margin/2, img_size[0])
        cropped = img[bb[1]:bb[3],bb[0]:bb[2],:]
        aligned = misc.imresize(cropped, (image_size, image_size), interp='bilinear')
        prewhitened = prewhiten(aligned)
        img_list.append(prewhitened)
        success_paths.append(img_path)
    images = np.stack(img_list)
    return success_paths, images

In [3]:
filenames = glob("photos/photos_raw/*/**")

In [4]:
success_filenames, aligned_images = load_and_align_data(filenames)

Creating networks and loading parameters


  0%|          | 0/94 [00:00<?, ?it/s]/home/julia/anaconda3/envs/MSDS603/lib/python3.6/site-packages/ipykernel_launcher.py:33: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/julia/anaconda3/envs/MSDS603/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
 55%|█████▌    | 52/94 [00:14<00:12,  3.26it/s]

no face found in photos/photos_raw/luke/UNADJUSTEDNONRAW_thumb_2446.jpg


100%|██████████| 94/94 [00:26<00:00,  3.88it/s]


In [5]:
for fname, img in zip(success_filenames, aligned_images):
    new_fname = fname.replace("photos_raw", "photos_pyalign").rsplit(".")[0] + ".png"
    os.makedirs(os.path.dirname(new_fname), exist_ok=True)
    print(fname)
    print(new_fname)
    print()
    misc.imsave(new_fname, img)

/home/julia/anaconda3/envs/MSDS603/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  import sys


photos/photos_raw/randy/UNADJUSTEDNONRAW_thumb_247e.jpg
photos/photos_pyalign/randy/UNADJUSTEDNONRAW_thumb_247e.png

photos/photos_raw/randy/UNADJUSTEDNONRAW_thumb_2462.jpg
photos/photos_pyalign/randy/UNADJUSTEDNONRAW_thumb_2462.png

photos/photos_raw/randy/UNADJUSTEDNONRAW_thumb_2484.jpg
photos/photos_pyalign/randy/UNADJUSTEDNONRAW_thumb_2484.png

photos/photos_raw/randy/UNADJUSTEDNONRAW_thumb_2481.jpg
photos/photos_pyalign/randy/UNADJUSTEDNONRAW_thumb_2481.png

photos/photos_raw/randy/UNADJUSTEDNONRAW_thumb_2463.jpg
photos/photos_pyalign/randy/UNADJUSTEDNONRAW_thumb_2463.png

photos/photos_raw/randy/UNADJUSTEDNONRAW_thumb_2470.jpg
photos/photos_pyalign/randy/UNADJUSTEDNONRAW_thumb_2470.png

photos/photos_raw/randy/UNADJUSTEDNONRAW_thumb_248b.jpg
photos/photos_pyalign/randy/UNADJUSTEDNONRAW_thumb_248b.png

photos/photos_raw/randy/UNADJUSTEDNONRAW_thumb_2473.jpg
photos/photos_pyalign/randy/UNADJUSTEDNONRAW_thumb_2473.png

photos/photos_raw/randy/UNADJUSTEDNONRAW_thumb_246b.jpg
photos/p

photos/photos_raw/julia/UNADJUSTEDNONRAW_thumb_2488.jpg
photos/photos_pyalign/julia/UNADJUSTEDNONRAW_thumb_2488.png

photos/photos_raw/julia/UNADJUSTEDNO

limit_output extension: Maximum message size of 10000 exceeded with 11134 characters

In [6]:
def load_image(img, do_random_crop, do_random_flip, image_size, do_prewhiten=True):
    images = np.zeros((1, image_size, image_size, 3))
    img = misc.imread(img)
    if img.ndim == 2:
        img = to_rgb(img)
    if do_prewhiten:
        img = prewhiten(img)
    img = crop(img, do_random_crop, image_size)
    img = flip(img, do_random_flip)
    images[:,:,:,:] = img
    return images

In [7]:
from collections import Counter

filenames = glob("photos/photos_pyalign/*/**")

with tf.Graph().as_default():
    with tf.Session() as sess:
        facenet.load_model("20180408-102900")

        # Get input and output tensors
        images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
        images_placeholder = tf.image.resize_images(images_placeholder,(160,160))
        embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
        phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")

        image_size = 160
        embedding_size = embeddings.get_shape()[1]
        extracted_dict = {}
        photo_counter = Counter()
        # Run forward pass to calculate embeddings
        for i, filename in enumerate(filenames):
            images = load_image(filename, False, False, image_size)
            feed_dict = { images_placeholder:images, phase_train_placeholder:False }
            feature_vector = sess.run(embeddings, feed_dict=feed_dict)
            name = filename.split("/")[-2]
            photo_counter[name] += 1
            extracted_dict["{}_{}".format(name, photo_counter[name])] =  feature_vector

        with open('extracted_dict.pickle','wb') as f:
            pickle.dump(extracted_dict,f)


Model directory: 20180408-102900
Metagraph file: model-20180408-102900.meta
Checkpoint file: model-20180408-102900.ckpt-90
INFO:tensorflow:Restoring parameters from 20180408-102900/model-20180408-102900.ckpt-90


/home/julia/anaconda3/envs/MSDS603/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
# for orig, img in zip(success_filenames, aligned_images):
#     plt.imshow(misc.imread(orig))
#     plt.axis('off')
#     plt.show()
#     plt.imshow(img)
#     plt.axis('off')
#     plt.show()